In [ ]:
QUERY_TIME = 1 #days
RETENTION_TIME = 28 #days
IMAGE_FOLDER = '/arcgis/home/images'

## Install and Important Package/Environment

In [ ]:
!pip install sentinelsat #Copernicus Hub API 
!pip install boto3 #AWS S3 API

In [ ]:
from arcgis.gis import GIS
from arcgis import geometry as geom
from arcgis.learn import FasterRCNN
from arcgis.features import Feature
from arcpy.ia import *
from arcgis.raster.functions import clip, extract_band
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt, SentinelProductsAPI, make_path_filter, SentinelProductsAPI
from datetime import datetime as dt, timedelta
from collections import OrderedDict
import os, glob, shutil, boto3, zipfile, json, arcpy

In [ ]:
gis = GIS('home')

In [ ]:
with open('/arcgis/home/notebook_config.txt','r') as infile:
    data = infile.read()
config = json.loads(data)

## Utility Functions and Classes

In [ ]:
def toShapely(poly):
    '''converts arcgis polygon JSON to shapely format'''
    pts = []
    for ring in poly['rings'][0]:
        pts.append(' '.join([str(pt) for pt in ring]))
    return "POLYGON(({}))".format(','.join(pts))

In [ ]:
def affinePoint(raster, x, y):
    '''basic affine function to convert pixels to lat/lon. Will only work on non-rotated, non-orthorectified imagery'''
    xpixel = raster.meanCellWidth
    ypixel = raster.meanCellHeight
    rotation = 0.0
    extent = str(raster.extent).split(' ')
    xmin = extent[0]
    ymax = extent[3]

    x = float(xpixel) * x + float(rotation) * y + float(xmin) #treat inputs as floats
    y = float(ymax) - (float(ypixel)* y + float(rotation) * x) #function transitions image coordiantes to geographic coordinates

    return x, y

In [ ]:
def upload_files(path,file):
    '''uploads image file to S3 bucket'''
    session = boto3.Session(
        aws_access_key_id=config['s3_access_key'],
        aws_secret_access_key=config['s3_secret_access_key'],
        region_name='us-east-2'
    )
    s3 = session.resource('s3')
    bucket = s3.Bucket('sentinel1-imagery')

    with open(path, 'rb') as data:
        outpath = path.replace(IMAGE_FOLDER,file)
        bucket.put_object(Key=outpath, Body=data)

In [ ]:
class SentinelFile():
    '''class to handle splitting Sentinel-1 file names into metadata'''
    def __init__(self,filename):
        values = filename.split('-')
        self.sourceimage = filename
        self.sensor = values[0].upper()
        self.modality = values[1].upper()
        self.productname = values[2].upper()
        self.datetime = dt.strptime(values[4].upper(),'%Y%m%dT%H%M%S')
        self.aoi = None
        self.country = None
        self.shipdetected = None
        self.geometry = None
    
    def toDict(self):
        return self.__dict__
    
    @staticmethod
    def splitSentinelName(filename):
        values = filename.split('-')
        output = [filename]
        output.append('Sentinel-1A') if values[0] == 's1a' else output.append('Sentinel-1B')
        output.append(values[1].upper())
        output.append(values[2].upper())
        output.append('Ground Range Detected')
        output.append(values[3].upper())
        output.append(dt.strptime(values[4].upper(),'%Y%m%dT%H%M%S'))
        return output
    
    def toFeature(self):
        geom = json.loads(self.geometry.JSON)
        attr = self.__dict__.copy()
        del attr['geometry']
        return Feature(geom,attr)   

In [ ]:
class Detection():
    '''class for handling each individual ship detected'''
    def __init__(self):
        self.detectclass = None
        self.confidence = None
        self.datetime = None
        self.aoi = None
        self.sourceimage = None
        self.sensor = None
        self.modality = None
        self.passid = None
        self.country = None
        self.productname = None
        self.geometry = None
        
    def toDict(self):
        return self.__dict__
    
    def toFeature(self):
        geom = json.loads(self.geometry.JSON)
        attr = self.__dict__.copy()
        attr['class'] = attr['detectclass']
        del attr['geometry']
        del attr['detectclass']
        return Feature(geom,attr)

In [ ]:
def ConvertPredictionsToDetections(detections,raster,sentinelFile,aoi,aoi_geom):
    '''function to map the results of the model.predict function to detection objects'''
    output = []
    positions = detections[0]
    classes = detections[1]
    confidences = detections[2]
    _sr4326 = arcpy.SpatialReference(4326)

    for ind,pos in enumerate(positions):
        xmin,ymin = affinePoint(raster,pos[0],pos[1])
        xmax,ymax = affinePoint(raster,pos[0]+pos[2],pos[1]+pos[3])
        polygon = arcpy.Polygon(arcpy.Array([
            arcpy.Point(xmin,ymin),
            arcpy.Point(xmax,ymin),
            arcpy.Point(xmax,ymax),
            arcpy.Point(xmin,ymax)
        ]),_sr4326)
        if not aoi_geom.disjoint(polygon): #check that the detction falls within the water polygon
            detect = Detection()
            detect.detectclass = classes[ind]
            detect.confidence = round(confidences[ind]*100,2)
            detect.datetime = sentinelFile.datetime
            detect.aoi = aoi
            detect.sourceimage = sentinelFile.sourceimage
            detect.sensor = sentinelFile.sensor
            detect.modality = sentinelFile.modality
            detect.passid = None
            detect.country = None
            detect.productname = sentinelFile.productname
            detect.geometry = polygon  
            output.append(detect)
    return output

## Clean Up Folders and Layers

In [ ]:
'''EMPTY IMAGE FOLDER ON SERVER'''
for filename in os.listdir(IMAGE_FOLDER):
    file_path = os.path.join(IMAGE_FOLDER, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

## Get AOI GeoJSON

In [ ]:
# Title: Ship Detection Geofences | Type: Feature Service
aois = gis.content.get("<PORTAL ITEM ID FOR FEATURE SERVICE WITH DETECTION AOIS>").layers[0]
aois

In [ ]:
wkts = []
featureset = aois.query(where="approved = 'Yes'",out_sr=4326) #check that AOIs are approved before tasking them
for f in featureset:
    poly = f.geometry
    wkts.append([f.get_value('name'),toShapely(poly)])
print("FOUND {} AOIS".format(len(wkts)))

## Query Copernicus Sentinel API

In [ ]:
api = SentinelAPI(config['copernicus_usr'],config['copernicus_pwd'])
products_api = SentinelProductsAPI(config['copernicus_usr'],config['copernicus_pwd'])

In [ ]:
d = dt.today() - timedelta(days=1) #copernicus has a slower upload data, so search for images from (today-2) to (today-1)
end = dt(d.year,d.month,d.day,23,59,59)
start = end - timedelta(days=QUERY_TIME)
start,end

In [ ]:
'''search copernicus for images over the AOIs'''
products = OrderedDict()
sources = {}
for w in wkts:
    result = api.query(w[1],raw='GRD',date=(start,end))
    for r in result.items():
        sources.update({r[1]['filename']:w[0]})
    products.update(result)
sources, len(products)

In [ ]:
'''download the images if they have VV polarisation'''
for p in products:
    if 'VV' in products[p]['polarisationmode']: #only download the image if VV polarisationis available
        print(p)
        products_api.download(p,directory_path=IMAGE_FOLDER)

In [ ]:
'''get a list of the full path to all downloaded images'''
imgs = []
for filename in glob.iglob(IMAGE_FOLDER + '**/*.zip', recursive=True):
     imgs.append(filename)
imgs

In [ ]:
'''unzip all images'''
for img in imgs:
    with zipfile.ZipFile(img,'r') as z:
        z.extractall(IMAGE_FOLDER)
    print('Extracted:',os.path.join(IMAGE_FOLDER,os.path.basename(imgs[0]).split('.')[0]))
    os.remove(img) #zip is no longer needed

## Run Ship Detection

In [ ]:
# Title: Ship Detections | Type: Feature Service 
detectLayer = gis.content.get("<PORTAL ITEM ID FOR OUTPUT DETECTION LAYER>").layers[0]
detectLayer

In [ ]:
'''get a list of all .tiffs to process'''
tiffs = []
for ind,f in enumerate(glob.glob(IMAGE_FOLDER + '/**/*vv*.tiff', recursive=True)):
    safefile = f.split('/')[4]
    tiffs.append([sources[safefile],f])
tiffs

In [ ]:
'''load model from EMD in notebook files, or use a DLPK'''
model = FasterRCNN.from_model(r'/arcgis/home/model/SARShipDetection.emd')
model

In [ ]:
detectObjects = []
feature_count = []
for aoi_name,tiff in tiffs:
    try:
        imgName = os.path.basename(tiff)
        print("Processing: {}, {}...".format(aoi_name,imgName))
        sentinelFile = SentinelFile(imgName) #create sentinelfile object to store file metadata

        #get extent of the aoi for clipping
        extent_aoi = aois.query(where="name = '{}'".format(aoi_name),out_sr=4326).features[0]
        extent_aoi = arcpy.AsShape(extent_aoi.geometry,True)

        #create a featureclass of the aoi for clipping
        if arcpy.Exists(r'in_memory/clip'): arcpy.Delete_management(r'in_memory/clip')
        clip_fc = arcpy.CreateFeatureclass_management(r'in_memory','clip','POLYGON',spatial_reference=arcpy.SpatialReference(4326))
        with arcpy.da.InsertCursor(clip_fc,['SHAPE@']) as cursor: cursor.insertRow([extent_aoi])
        print('Clip_fc created...')

        #clip raster to processing extent
        clipPath = r'/arcgis/home/images/clip_temp.tif'
        if os.path.isfile(clipPath): os.remove(clipPath)
        arcpy.management.Clip(tiff, clip_fc,clipPath, None, "65536","NONE","NO_MAINTAIN_EXTENT")
        clipped_raster = arcpy.Raster(clipPath)
        print('Clip complete...')

        resizePath = r'/arcgis/home/images/resize_temp.tif'
        if os.path.isfile(resizePath): os.remove(resizePath)
        #resize and set pixel depth to match model requirements
        resized_raster = arcpy.management.CopyRaster(clipped_raster, resizePath, '', None, "65536", "NONE", "ColormapToRGB", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")
        resized_raster = arcpy.Raster(resizePath)
        del clipped_raster
        print('Resize complete...')

        #save for detecting
        save_path = os.path.join(IMAGE_FOLDER,os.path.basename(sentinelFile.sourceimage).split('.')[0] + '_clipped.tif')
        if os.path.exists(save_path): os.remove(save_path)
        resized_raster.save(save_path)

        #run inferencing
        print('Running inferencing...')
        detections = model.predict(save_path,return_scores=True)

        #run affine function and create detect objects
        detectObjects = ConvertPredictionsToDetections(detections,resized_raster,sentinelFile,aoi_name,extent_aoi)
        toAdd = [d.toFeature() for d in detectObjects]
        detectLayer.edit_features(adds=toAdd)
        del resized_raster
        feature_count.append(len(detectObjects))
        print("DETECTED {} SHIPS".format(len(detectObjects)))
    except Exception as e:
        tiffs.remove([aoi_name,tiff])
        print(e)

## Update Mosaic

In [ ]:
'''If you want to store imagery in S3 and an Image Service, use these cells below'''
sde_path = r'<PATH TO MOSAIC DATASET IN SDE>'
acs_path = r'<PATH TO ACS (CLOUD CONNECTION FILE) FILE IN NOTEBOOK FILES>'
sde_path, acs_path

In [ ]:
for tif in tiffs:
    filename = os.path.basename(tif[1]).split('.')[0] + '_clipped.tif' #only store clipped rasters to save some bits
    print('Uploading {}...'.format(filename))
    sentinelFile = SentinelFile(filename)
    upload_files(os.path.join(IMAGE_FOLDER,filename),dt.strftime(sentinelFile.datetime,'%m-%d-%Y'))

In [ ]:
arcpy.env.parallelProcessingFactor = 0
arcpy.env.addOutputsToMap = False
arcpy.env.workspace = sde_path
mdname = sde_path
rastype = "Raster Dataset"
inpath = acs_path
updatecs = "UPDATE_CELL_SIZES"
updatebnd = "UPDATE_BOUNDARY"
updateovr = "NO_OVERVIEWS"
maxlevel = None
maxcs = 0
maxdim = 1500
spatialref = None
inputdatafilter = "*.tif"
subfolder = "SUBFOLDERS"
duplicate = "EXCLUDE_DUPLICATES"
buildpy = "NO_PYRAMIDS"
calcstats = "CALCULATE_STATISTICS"
buildthumb = "NO_THUMBNAILS"
comments = "Add Raster Datasets"
forcesr = "NO_FORCE_SPATIAL_REFERENCE"
estimatestats = "NO_STATISTICS"
auxilaryinput = None
enablepixcache = "NO_PIXEL_CACHE"
cachelocation = ""

arcpy.AddRastersToMosaicDataset_management( 
    mdname,  rastype, inpath, updatecs, updatebnd, updateovr,
     maxlevel, maxcs, maxdim, spatialref, inputdatafilter,
     subfolder, duplicate, buildpy, calcstats, 
     buildthumb, comments, forcesr, estimatestats,
     auxilaryinput, enablepixcache, cachelocation)

In [ ]:
# Title: CollectionFootprints | Type: Feature Service
footprints = gis.content.get("<PORTAL ITEM ID FOR FEATURE SERVICE WITH IMAGE FOOTPRINTS>").layers[0]
footprints

In [ ]:
'''add image footprints and metadata to footprints service'''
count = 0
for aoi,tiff in tiffs:
    print('Adding {}...'.format(tiff))
    filename = os.path.basename(tiff).split('.')[0] + '_clipped.tif'
    fData = SentinelFile.splitSentinelName(filename)
    with arcpy.da.UpdateCursor(sde_path,['name','SensorName','BeamMode','tag','ProductName','Polarization','AcquisitionDate','SHAPE@'],"name = '{}'".format(filename.split('.')[0])) as cursor:
        for row in cursor:
            row[:-1] = fData
            cursor.updateRow(row)
            
    sentinelFile = SentinelFile(filename)
    raster = arcpy.Raster(os.path.join(IMAGE_FOLDER,filename))
    extent = str(raster.extent).split(' ')
    xmin,ymin,xmax,ymax = extent[0],extent[1],extent[2],extent[3]
    polygon = arcpy.Polygon(arcpy.Array([
        arcpy.Point(xmin,ymin),
        arcpy.Point(xmax,ymin),
        arcpy.Point(xmax,ymax),
        arcpy.Point(xmin,ymax)
    ]),arcpy.SpatialReference(4326))
    sentinelFile.geometry = polygon
    sentinelFile.aoi = aoi
    try: 
        sentinelFile.shipdetected = feature_count[count] 
    except Exception as e:
        print(e)
        pass
    footprints.edit_features(adds=[sentinelFile.toFeature()])
    count += 1

## Clean Up

In [ ]:
'''EMPTY IMAGE FOLDER ON SERVER'''
for filename in os.listdir(IMAGE_FOLDER):
    file_path = os.path.join(IMAGE_FOLDER, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
'''remove old rasters from mosaic dataset'''
retain_time = dt.now() - timedelta(days=RETENTION_TIME) #6hr time dif and 6hrs to handle UTC conversion
query = "acquisitiondate < TIMESTAMP '{}'".format(retain_time.strftime('%Y-%m-%d %H:%M:%S'))
arcpy.management.RemoveRastersFromMosaicDataset(sde_path,query,'UPDATE_BOUNDARY','MARK_OVERVIEW_ITEMS','DELETE_OVERVIEW_IMAGES','DELETE_ITEM_CACHE')

In [ ]:
'''manually clean old rows from footprint data'''
retain_time = dt.now() - timedelta(days=RETENTION_TIME) #6hr time dif and 6hrs to handle UTC conversion
query = "datetime < TIMESTAMP '{}'".format(retain_time.strftime('%Y-%m-%d %H:%M:%S'))
try:
    result = footprints.delete_features(where=query)
    print('Features Deleted:',len(result['deleteResults']))
    print('---------------------------------------------')
    print('Features Failed to Delete:')
    for r in result['deleteResults']:
        if r['success'] == False:
            print(r)
except Exception as e:
    print(e)

In [ ]:
'''remove old images from s3'''
count = 0
session = boto3.Session(
    aws_access_key_id=config['s3_access_key'],
    aws_secret_access_key=config['s3_secret_access_key'],
    region_name='us-east-2'
)
s3 = session.resource('s3')
bucket = s3.Bucket('sentinel1-imagery')
for obj in bucket.objects.all():
    prefix = obj.key.split('/')[0]
    try:
        dtg = dt.strptime(prefix,'%m-%d-%Y')
        if dtg < dt.now() - timedelta(days=RETENTION_TIME):
            obj.delete()
            count += 1
    except:
        pass
        
print('Removed {} images...'.format(count))